<a href="https://colab.research.google.com/github/HoTrungQuan2004/ParkingAreaManage/blob/main/Process_numberplate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyrebase4 --quiet

In [ ]:
import pyrebase
import time
from datetime import datetime

In [ ]:
# --- Cấu hình Firebase ---
config = {
    "apiKey": "<YOUR_FIREBASE_API_KEY>",
    "authDomain": "<YOUR_PROJECT_ID>.firebaseapp.com",
    "databaseURL": "https://liencesplayeai-default-rtdb.firebaseio.com",
    "storageBucket": "<YOUR_PROJECT_ID>.appspot.com"
}

firebase = pyrebase.initialize_app(config)
db = firebase.database()

# --- Cấu hình số slot ---
TOTAL_SLOTS = 100

# --- Các node trên Firebase ---
NUMBERPLATE_NODE = "numberplate"
PARKING_NODE = "vehicles"          # Danh sách xe đang đỗ (biển số -> thời gian vào)
STATS_NODE = "parking_stats"       # Thống kê slot
DAILY_NODE = "daily_counts"
MONTHLY_NODE = "monthly_counts"

# --- Bộ nhớ tạm thời trên RAM ---
current_vehicles = {}  # {biển_số: thời_gian_đỗ}

def get_today():
    return datetime.now().strftime("%Y-%m-%d")

def get_month():
    return datetime.now().strftime("%Y-%m")

def update_stats():
    """Cập nhật số slot trống/đã dùng lên Firebase"""
    occupied = len(current_vehicles)
    stats = {
        "total_slots": TOTAL_SLOTS,
        "occupied_slots": occupied,
        "available_slots": TOTAL_SLOTS - occupied,
        "last_updated": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    db.child(STATS_NODE).set(stats)

def update_daily_count():
    """Tăng lượt xe hôm nay"""
    today = get_today()
    node = f"{DAILY_NODE}/{today}"
    cur = db.child(node).get().val()
    count = cur.get("count", 0) + 1 if cur else 1
    db.child(node).set({"date": today, "count": count})

def update_monthly_count():
    """Tăng lượt xe tháng này"""
    month = get_month()
    node = f"{MONTHLY_NODE}/{month}"
    cur = db.child(node).get().val()
    count = cur.get("count", 0) + 1 if cur else 1
    db.child(node).set({"month": month, "count": count})

def process_plate(plate, date_time, key):
    plate = plate.strip().upper()
    if not plate or len(plate) < 4:
        print(f"[WARN] Plate không hợp lệ: {plate}")
        return

    # --- Xe ĐI RA (đã từng vào) ---
    if plate in current_vehicles:
        print(f"[INFO] 🚗 Xe RA: {plate} ({current_vehicles[plate]}) -> {date_time}")
        # Xóa khỏi danh sách xe đang đỗ trên server
        db.child(PARKING_NODE).child(plate).remove()
        del current_vehicles[plate]
        # Cập nhật slot
        update_stats()
        # Có thể log lịch sử ra vào ở đây nếu muốn
    else:
        # --- Xe ĐI VÀO ---
        print(f"[INFO] 🚗 Xe VÀO: {plate} @ {date_time}")
        current_vehicles[plate] = date_time
        db.child(PARKING_NODE).child(plate).set({"entry_time": date_time})
        update_stats()
        update_daily_count()
        update_monthly_count()

    # Xóa entry đã xử lý khỏi queue
    db.child(NUMBERPLATE_NODE).child(key).remove()

def sync_current_vehicles():
    """Đồng bộ danh sách xe từ Firebase vào RAM khi bắt đầu chạy"""
    global current_vehicles
    data = db.child(PARKING_NODE).get().val()
    if data:
        current_vehicles = {k: v["entry_time"] if isinstance(v, dict) and "entry_time" in v else "" for k, v in data.items()}
    else:
        current_vehicles = {}

def main_loop(poll_interval=3):
    print("---- Parking Manager Started ----")
    sync_current_vehicles()
    update_stats()
    while True:
        # Đọc danh sách biển số mới gửi lên
        plates = db.child(NUMBERPLATE_NODE).get().val()
        if plates:
            for key, value in plates.items():
                plate = value.get("number_plate", "").strip()
                date_time = value.get("date_time", "")
                process_plate(plate, date_time, key)
        else:
            # print("No new plates.")
            pass
        time.sleep(poll_interval)

if __name__ == "__main__":
    main_loop(poll_interval=3)  # Kiểm tra mỗi 3 giây, có thể tăng/giảm tùy nhu cầu